# 스파크 스트리밍 실습 : 카프카 스트리밍 애플리케이션

> 플루언트디를 통해 생성된 `movies` 토픽의 메시지를 스파크 스트리밍 애플리케이션을 통해 카프카 토픽을 처리합니다

## 학습 목표
* `Kafka`에 저장된 스트리밍 데이터를 처리합니다
  - 카프카 메시지 프로듀서를 통해서 카프카에 스트림 데이터를 생성합니다


In [1]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from IPython.display import display, display_pretty, clear_output, JSON

spark = (
    SparkSession
    .builder
    .config("spark.sql.session.timeZone", "Asia/Seoul")
    .getOrCreate()
)

# 노트북에서 테이블 형태로 데이터 프레임 출력을 위한 설정을 합니다
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # display enabled
spark.conf.set("spark.sql.repl.eagerEval.truncate", 100) # display output columns size

# 공통 데이터 위치
home_jovyan = "/home/jovyan"
work_data = f"{home_jovyan}/work/data"
work_dir=!pwd
work_dir = work_dir[0]

# 로컬 환경 최적화
spark.conf.set("spark.sql.shuffle.partitions", 5) # the number of partitions to use when shuffling data for joins or aggregations.
spark.conf.set("spark.sql.streaming.forceDeleteTempCheckpointLocation", "true")

# 현재 기동된 스파크 애플리케이션의 포트를 확인하기 위해 스파크 정보를 출력합니다
spark

In [2]:
# 스트림 테이블을 주기적으로 조회하는 함수 (name: 이름, sql: Spark SQL, iterations: 반복횟수, sleep_secs: 인터벌)
def displayStream(name, sql, iterations, sleep_secs):
    from time import sleep
    i = 1
    for x in range(iterations):
        clear_output(wait=True)              # 출력 Cell 을 지웁니다
        display('[' + name + '] Iteration: '+str(i)+', Query: '+sql)
        display(spark.sql(sql))              # Spark SQL 을 수행합니다
        sleep(sleep_secs)                    # sleep_secs 초 만큼 대기합니다
        i += 1

# 스트림 쿼리의 상태를 주기적으로 조회하는 함수 (name: 이름, query: Streaming Query, iterations: 반복횟수, sleep_secs: 인터벌)
def displayStatus(name, query, iterations, sleep_secs):
    from time import sleep
    i = 1
    for x in range(iterations):
        clear_output(wait=True)      # Output Cell 의 내용을 지웁니다
        display('[' + name + '] Iteration: '+str(i)+', Status: '+query.status['message'])
        display(query.lastProgress)  # 마지막 수행된 쿼리의 상태를 출력합니다
        sleep(sleep_secs)            # 지정된 시간(초)을 대기합니다
        i += 1

In [4]:
filePath = f"{work_data}/json"
fileJson = spark.read.json(filePath)
fileJson.printSchema() # 기본적으로 스키마를 추론하는 경우 integer 가 아니라 long 으로 잡는다
display(fileJson)

root
 |-- emp_id: long (nullable = true)
 |-- emp_name: string (nullable = true)



emp_id,emp_name
1,엘지전자
2,엘지화학
3,엘지에너지솔루션
100,엘지


In [5]:
filePath = f"{work_data}/json"
queryName = "fileSource"
checkpointLocation = f"{work_dir}/tmp/{queryName}"
!rm -rf $checkpointLocation
fileSchema = (
    StructType()
    .add(StructField("emp_id", LongType()))
    .add(StructField("emp_name", StringType()))
)
fileReader = (
    spark
    .readStream
    .format("json")
    .schema(fileSchema)
    .load(filePath)
)
fileSelector = fileReader.select("emp_id", "emp_name")
fileWriter = (
    fileSelector
    .writeStream
    .queryName(queryName)
    .format("memory")
    .outputMode("update")
)
fileTrigger = (
    fileWriter
    .trigger(processingTime="1 second")
    .option("checkpointLocation", checkpointLocation)
)
fileQuery = fileTrigger.start()
displayStream("fileSource", f"select * from {queryName}", 3, 3)
fileQuery.stop()

'[fileSource] Iteration: 3, Query: select * from fileSource'

emp_id,emp_name
1,엘지전자
2,엘지화학
3,엘지에너지솔루션
100,엘지


## Kafka 소스 스트리밍 실습

#### 1. 카프카에 데이터 쓰기 - producer.py
```python
#!/usr/bin/env python

import sys
from time import sleep
from json import dumps
import json
from kafka import KafkaProducer
import names

# 카프카에 데이터를 전송하는 코드
def produce(port):
    try:
        hostname="kafka:%d" % port
        producer = KafkaProducer(bootstrap_servers=[hostname],
            value_serializer=lambda x: dumps(x).encode('utf-8')
        )
        data = {}
        for seq in range(9999):
            print("Sequence", seq)
            first_name = names.get_first_name()
            last_name = names.get_last_name()
            data["first"] = first_name
            data["last"] = last_name
            producer.send('events', value=data)
            sleep(0.5)
    except:
        import traceback
        traceback.print_exc(sys.stdout)

# 카프카의 데이터 수신을 위한 내부 포트는 9093
if __name__ == "__main__":
    port = 9093
    if len(sys.argv) == 2:
        port = int(sys.argv[1])
    produce(port)
```
#### 2. 터미널에 접속하여 카프카 메시지를 생성합니다"
```bash
(base) # python producer.py
```

#### 3. 스파크를 통해 생성된 스트리밍 데이터를 확인합니다

In [ ]:
# 컨테이너 내부에서 토픽에 접근하기 위한 포트는 9093
kafkaReader = (
    spark
    .readStream
    .format("kafka")
    .option("kafka.bootstrap.servers", "kafka:9093")
    .option("subscribe", "movies")
    .load()
)

kafkaSchema = (
    StructType()
    .add(StructField("movie", StringType()))
    .add(StructField("title", StringType()))
    .add(StructField("title_eng", StringType()))
    .add(StructField("year", IntegerType()))
    .add(StructField("grade", StringType()))
)
kafkaSelector = (
    kafkaReader
    .select(
        from_json(col("value").cast("string"), kafkaSchema).alias("movies")
    )
    .selectExpr("movies.title as title", "movies.year as year")
    .groupBy("year")
    .count().alias("count")
)

queryName = "kafkaSource"
kafkaWriter = (
    kafkaSelector
    .writeStream
    .queryName(queryName)
    .format("memory")
    .outputMode("complete")
)

checkpointLocation = f"{work_dir}/tmp/{queryName}"
!rm -rf $checkpointLocation
kafkaTrigger = (
    kafkaWriter
    .trigger(processingTime="1 second")
    .option("checkpointLocation", checkpointLocation)
)
kafkaQuery = kafkaTrigger.start()
displayStream("kafkaSource", f"select year, count from {queryName} order by count desc limit 5", 30, 3)
kafkaQuery.stop()

NameError: name 'spark' is not defined

In [7]:
kafkaQuery.status
kafkaQuery.stop()

In [3]:
# 컨테이너 내부에서 토픽에 접근하기 위한 포트는 9093
kafkaReader = (
    spark
    .readStream
    .format("kafka")
    .option("kafka.bootstrap.servers", "kafka:9093")
    .option("subscribe", "movies")
    .load()
)

kafkaSchema = (
    StructType()
    .add(StructField("movie", StringType()))
    .add(StructField("title", StringType()))
    .add(StructField("title_eng", StringType()))
    .add(StructField("year", IntegerType()))
    .add(StructField("grade", StringType()))
)

kafkaSelector = (
    kafkaReader
    .select(
        from_json(col("value").cast("string"), kafkaSchema).alias("movies")
    )
    .selectExpr("movies.movie as id", "movies.title as title", "movies.grade as grade", "movies.year as year")
)

queryName = "kafkaSink"
kafkaWriter = (
    kafkaSelector
    .writeStream
    .format("kafka")
    .option("kafka.bootstrap.servers", "kafka:9093")
    .option("topic", "kor_movies")
)

checkpointLocation = f"{work_dir}/tmp/{queryName}"
!rm -rf $checkpointLocation
kafkaTrigger = (
    kafkaWriter
    .trigger(processingTime="1 second")
    .option("checkpointLocation", checkpointLocation)
)
kafkaQuery = kafkaTrigger.start()

In [4]:
kafkaQuery.status

{'message': 'Terminated with exception: org/apache/spark/sql/internal/connector/SupportsStreamingUpdate',
 'isDataAvailable': False,
 'isTriggerActive': False}